# Carga do modelo, conexão com o banco, execução e output da classificação de volta para o banco
### Equipe 5 - Safra Hackers



## Declaração de funções

In [198]:
def insert_data(mycursor_,result_,clusters_,ldict_):
    for i in range(len(result_)):
        sql = "INSERT INTO safra.modelo (conta, saida,entrada,cluster) VALUES ('{0}',{1},{2},'{3}')".format(result[i][0],result[i][1],result[i][2],ldict_[str(clusters_[i])])
        val = ("John", "Highway 21")
        mycursor_.execute(sql)
        mydb.commit()
        
def create_connection(host_,user_,password_,database_):
    conn = mysql.connector.connect(
      host=host_,
      user=user_,
      password=password_,
      database=database_
    )
    return conn

## Declarando Bibliotecas

In [166]:
import mysql.connector
import pickle
import json 

from sklearn.cluster import KMeans
from configparser import ConfigParser

## Lendo arquivo de configuração e conectando ao banco de dados

In [189]:
config_object = ConfigParser()
config_object.read("config.ini")

['config.ini']

In [ ]:
mydb = create_connection(host_=config_object['CONN_STRING']["host"],
                         user_=config_object['CONN_STRING']["user"],
                         password_=config_object['CONN_STRING']["password"],
                         database_=config_object['CONN_STRING']["database"]
                        )

### Leitura dos dados de conta e balanços

In [107]:
cursor = mydb.cursor()
cursor.execute("SELECT conta,saida,entrada FROM safra.transferencias")

In [108]:
result = cursor.fetchall()

for x in result:
    print(x)

('C01', 450.0, 500.0)
('C02', 1680.0, 1500.0)


## Preparando entrada do modelo

In [111]:
kmeans_input = [[res[1],res[2]] for res in result]

## Carga do modelo e predição

In [112]:
loaded_model = pickle.load(open('Kmeans.sav', 'rb'))

In [119]:
clusters = loaded_model.predict(kmeans_input)

## Inserção dos resultados de volta na base

In [194]:
#dicionário da indicação do kmeans
ldict = dict(config_object._sections["CLUSTERS"])

In [200]:
mycursor = mydb.cursor()

insert_data(mycursor,result,clusters,ldict)